In [ ]:

pip install torchsummary


In [ ]:
# Project 6: U-Net and Autoencoders

# Imports
import torch
from torchvision.datasets import MNIST
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn
from torchsummary import summary
from torch.optim import Adam
import numpy as np
import matplotlib.pyplot as plt
import time

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

# Global noise values
MILD = .1
MEDIUM = .25
SEVERE = .4


In [ ]:
class UNet(nn.Module):

    """
        Our UNet is a neural network that reduces an images to core features
        (while reducing the size of the image and increasing the channels),
        and then increases it back to its original size.

        Since the input images from MNIST are size 28x28, we did not want to
        fuclass UNet(nn.Module):
    def __init__(self, channels=3, output_classes=10):
        super().__init__()
        self.loss_func = nn.MSELoss()
        self.ConvLayer1 = self.ConvLayer(channels, 20, 3)
        self.MaxPool = nn.MaxPool2d(2)
        self.ConvLayer2 = self.ConvLayer(20, 40, 3)
        self.ConvLayer3 = self.ConvLayer(40, 80, 3)
        self.TransposeLayer1 = self.TransposeConvLayer(80, 40, 2)
        self.ConvLayer4 = self.ConvLayer(80, 40, 3)
        self.TransposeLayer2 = self.TransposeConvLayer(40, 20, 2)
        self.ConvLayer5 = self.ConvLayer(40, 20, 3)
        self.ConvOneByOne = nn.Conv2d(20, 3, kernel_size=1)  # Output should have 3 channelslly replicate the original UNet. Because we are simplifying the problem,
        a reduction in channel dimension makes sense here, as well as a reduction
        in total layers.

        We decided our architecture would look like this:
          - 1 CL (Convolutional Layer) followed by a Max Pool Layer 2 times, then
          - A bottom CL followed by a TCL (Transpose CL) 2 times, followed by
          - Two CL (the last one reducing the dimension size back to 1)

        The image gets reduced from a size of 28x28 -> 14x14 -> 7x7, the channel
        size gets increased from 1 -> 20 -> 40 -> 80, and then the size scales
        back up and the channel size scales back down.

        One key aspect of the UNet is that before the MaxPool layers, a copy of
        the outputs get saved so it can be concatenated later in the re-scaling side.
    """

    def __init__(self, channels=3, output_classes=10):
        super().__init__()
        self.loss_func = nn.MSELoss()
        self.ConvLayer1 = self.ConvLayer(channels, 20, 3)
        self.MaxPool = nn.MaxPool2d(2)
        self.ConvLayer2 = self.ConvLayer(20, 40, 3)
        self.ConvLayer3 = self.ConvLayer(40, 80, 3)
        self.TransposeLayer1 = self.TransposeConvLayer(80, 40, 2)
        self.ConvLayer4 = self.ConvLayer(80, 40, 3)
        self.TransposeLayer2 = self.TransposeConvLayer(40, 20, 2)
        self.ConvLayer5 = self.ConvLayer(40, 20, 3)
        self.ConvOneByOne = nn.Conv2d(20, 3, kernel_size=1)  # Output should have 3 channels

    # Remainder of the class stays the same


    def ConvLayer(self, in_ch, out_ch, kernel_size=3):
        sequence = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size, 1, 1),
            nn.ReLU())
        return sequence

    def TransposeConvLayer(self, in_ch, out_ch, kernel_size=2):
        sequence = nn.Sequential(
            nn.ConvTranspose2d(in_ch, out_ch, kernel_size, 2),
            nn.ReLU())
        return sequence

    def forward(self, x):
        a = self.ConvLayer1(x)
        x = a
        b = self.ConvLayer2(self.MaxPool(x))
        x = b
        x = self.TransposeLayer1(self.ConvLayer3(self.MaxPool(x)))
        x = self.TransposeLayer2(self.ConvLayer4(torch.cat([b,x], dim=1)))
        return self.ConvOneByOne(self.ConvLayer5(torch.cat([a,x], dim=1)))

    def num_parameters(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)


# Create a UNet instance and visualize parameter grid
unet = UNet().to(device)
print(f"Total parameters for UNet: {unet.num_parameters()}")
summary(unet, (3, 28, 28))


In [ ]:
class Autoencoder(nn.Module):

    """
        Our Autoencoder network is essentially the same thing as our UNet.
        However, we do not employ skip connections (we don't save the output
        of the Convolutional Layers in order to do concatenations later).

        The autoencoder employs the same architecture and size reduction/
        scaling sizes as our previous UNet architecture.
    """

    def __init__(self, channels=3, output_classes=10):
        super().__init__()
        self.loss_func = nn.MSELoss()
        self.layers = nn.Sequential(
            self.ConvLayer(channels, 20, 3),
            nn.MaxPool2d(2),
            self.ConvLayer(20, 40, 3),
            nn.MaxPool2d(2),
            self.ConvLayer(40, 80, 3),
            self.TransposeConvLayer(80, 40, 2),
            self.TransposeConvLayer(40, 20, 2),
            nn.Conv2d(20, 3, kernel_size=1)  # Output should have 3 channels
        )


    # ConvLayer and TransposeConvLayer methods remain the same

    def ConvLayer(self, in_ch, out_ch, kernel_size=3):
        return nn.Sequential(nn.Conv2d(in_ch, out_ch, kernel_size, 1, 1),
                             nn.ReLU())

    def TransposeConvLayer(self, in_ch, out_ch, kernel_size=2):
        return nn.Sequential(nn.ConvTranspose2d(in_ch, in_ch, kernel_size, 2),
                             nn.ReLU(),
                             nn.Conv2d(in_ch, out_ch, 3, 1, 1),
                             nn.ReLU())
    def forward(self, x):
        return self.layers(x)

    def num_parameters(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)



# Create an autoencoder instance and visualize parameter grid
autoencoder = Autoencoder().to(device)
print(f"Total parameters for Autoencoder: {autoencoder.num_parameters()}")
summary(autoencoder, (3, 28, 28))

In [ ]:
import os  # Add this import at the beginning of your script


In [ ]:
import numpy as np

def noisy_generator(batches, noise_type='salt_and_pepper'):
    for batch_x, batch_y in batches:
        if noise_type == 'salt_and_pepper':
            # Generate salt and pepper noise
            salt_vs_pepper = 0.3
            noise = np.random.choice([0, 1, 2], size=batch_x.shape, p=[1 - salt_vs_pepper, salt_vs_pepper / 2., salt_vs_pepper / 2.])
            salt_mask = noise == 1
            pepper_mask = noise == 2
            
            # Add salt noise
            salt_intensity = 1
            batch_noisy = batch_x.copy()
            batch_noisy[salt_mask] = salt_intensity
            
            # Add pepper noise
            pepper_intensity = 0
            batch_noisy[pepper_mask] = pepper_intensity
        elif noise_type == 'gaussian':
            # Add Gaussian noise
            mean = 0.
            std = 0.5
            batch_noisy = batch_x + np.random.normal(mean, std, size=batch_x.shape)
            batch_noisy = np.clip(batch_noisy, 0., 1.)
        elif noise_type == 'rayleigh':
            # Add Rayleigh noise
            scale = 0.1
            noise = np.random.rayleigh(scale, size=batch_x.shape)
            batch_noisy = batch_x + noise
            batch_noisy = np.clip(batch_noisy, 0., 1.)
        else:
            raise ValueError("Invalid noise type. Choose from 'salt_and_pepper', 'gaussian', or 'rayleigh'.")

        yield (batch_noisy, batch_y)


In [ ]:
from torchvision.transforms import Resize
from PIL import Image
import torch
from torch.utils.data import Dataset
import tensorflow as tf

class AddGaussianNoise(object):
    def __init__(self, mean=0., std=0.5):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean

class AddSaltAndPepperNoise(object):
    def __init__(self, prob=0.4):
        self.prob = prob
        
    def __call__(self, tensor):
        noisy_image = tensor.clone()
        salt_pepper = torch.rand_like(tensor)
        noisy_image[salt_pepper < self.prob / 2] = 0.0
        noisy_image[salt_pepper > 1 - self.prob / 2] = 1.0
        return noisy_image

class AddRayleighNoise(object):
    def __init__(self, scale=0.3):
        self.scale = scale
        
    def __call__(self, tensor):
        noise = torch.randn_like(tensor, dtype=torch.float) * self.scale
        noisy_image = tensor + noise
        noisy_image = torch.clamp(noisy_image, 0, 1)
        return noisy_image
class AddSpeckleNoise(object):
    def __init__(self, mean=0., std=0.5):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        noise = tf.random.normal(tf.shape(tensor), mean=self.mean, stddev=self.std, dtype=tf.float32)
        noisy_tensor = tensor + tensor * noise
        return noisy_tensor

mu=0.2
class CustomDataset(Dataset):
    def __init__(self, data_dir, transform=None, noise_type='gaussian'):
        self.data_dir = data_dir
        self.transform = transform
        self.image_paths = []
        self.noise_type = 'gaussian'

        for root, _, files in os.walk(data_dir):
            for file in files:
                if file.endswith(".jpg") or file.endswith(".jpeg") or file.endswith(".png"):
                    self.image_paths.append(os.path.join(root, file))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert("RGB")
        
        if self.transform:
            image = self.transform(image)

        # Add noise to the image here based on the specified noise type
        if self.noise_type == 'gaussian':
            noise_func = AddGaussianNoise()
        elif self.noise_type == 'salt_and_pepper':
            noise_func = AddSaltAndPepperNoise()
        elif self.noise_type == 'rayleigh':
            noise_func = AddRayleighNoise()
        elif self.noise_type == 'speckle':
            # Add Gaussian Noise
            noise_func = AddSpeckleNoise(std=0.5)
        else:
            raise ValueError("Unsupported noise type")

        noisy_image = noise_func(image)
        
        return noisy_image, image  # Return both the noisy image and the clean image


In [ ]:


transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize images to 32x32 (adjust as needed)
    transforms.ToTensor()         # Convert images to tensors
])

data_dir = "/kaggle/input/dataset-explo/bsds500/images/train/all"
custom_dataset = CustomDataset(data_dir, transform=transform)

# Check the length of the dataset
print(len(custom_dataset))

# Create a DataLoader for your custom dataset
batch_size = 32
custom_dataloader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

# Train function for both models
def train(model, dataloader, epochs, optimizer):
    model.train()
    criterion = nn.MSELoss()
    for epoch in range(epochs):
        epoch_loss = 0.0
        for noisy_image, clean_image in dataloader:
            optimizer.zero_grad()
            input_data = noisy_image.to(device)
            output_data = model(input_data)
            loss = criterion(output_data, clean_image.to(device))
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * input_data.size(0)
        epoch_loss /= len(dataloader.dataset)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")

# Example usage
num_epochs = 100
lr = 0.002
u_net = UNet().to(device)
autoencoder = Autoencoder().to(device)
u_opt = Adam(u_net.parameters(), lr=lr)
auto_opt = Adam(autoencoder.parameters(), lr=lr)

# Train U-Net
train(u_net, custom_dataloader, num_epochs, u_opt)

# Train Autoencoder
train(autoencoder, custom_dataloader, num_epochs, auto_opt)




In [ ]:
from torchvision.transforms import Resize

resize_transform = Resize((256, 256))


In [ ]:
#  Create an instance of your custom dataset
test_data_dir = '/kaggle/input/aaaaaaaaaaa/owndataset/all'
test_dataset = CustomDataset(test_data_dir, transform=transforms.Compose([resize_transform, transforms.ToTensor()]))

# Create a DataLoader for the test dataset
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:

#  Create an instance of your custom dataset
test_data_dir = "/kaggle/input/aaaaaaaaaaa/owndataset/all"
test_dataset = CustomDataset(test_data_dir, transform=transforms.Compose([resize_transform, transforms.ToTensor()]))

# Create a DataLoader for the test dataset
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

def test_model(model, model2, dataloader):
    model.eval()  # Set the model to evaluation mode
    model2.eval()  # Set the model to evaluation mode

    with torch.no_grad():  # Disable gradient tracking during inference
        for i, (noisy_images, clean_images) in enumerate(dataloader):
            input_data = noisy_images.to(device)
            output_data = model(input_data)
            output_data2 = model2(input_data)

            for j in range(len(noisy_images)):  # Iterate over each image in the batch
                # Calculate PSNR and SSIM for model 1
                psnr_value = psnr(clean_images[j].permute(1, 2, 0).cpu().numpy(), output_data[j].permute(1, 2, 0).cpu().numpy())
                ssim_value = ssim(clean_images[j].permute(1, 2, 0).cpu().numpy(), output_data[j].permute(1, 2, 0).cpu().numpy(), win_size=3, data_range=1, multichannel=True)
                print(f"Model 1 - PSNR: {psnr_value:.4f}, SSIM: {ssim_value:.4f}")

                # Calculate PSNR and SSIM for model 2
                psnr_value2 = psnr(clean_images[j].permute(1, 2, 0).cpu().numpy(), output_data2[j].permute(1, 2, 0).cpu().numpy())
                ssim_value2 = ssim(clean_images[j].permute(1, 2, 0).cpu().numpy(), output_data2[j].permute(1, 2, 0).cpu().numpy(), win_size=3, data_range=1, multichannel=True)
                print(f"Model 2 - PSNR: {psnr_value2:.4f}, SSIM: {ssim_value2:.4f}")

                # Display images and metrics (you can add your metric calculation here)
                # For example, displaying the images
                input_image = noisy_images[j].permute(1, 2, 0).cpu().numpy()
                denoised_image = output_data[j].permute(1, 2, 0).cpu().numpy()
                denoised_image2 = output_data2[j].permute(1, 2, 0).cpu().numpy()

                fig, axs = plt.subplots(1, 3, figsize=(12, 4))
                axs[0].imshow(input_image)
                axs[0].set_title("Noisy Image")
                axs[0].axis("off")
                axs[1].imshow(denoised_image)
                axs[1].set_title("Denoised Image (Model 1)")
                axs[1].axis("off")
                axs[2].imshow(denoised_image2)
                axs[2].set_title("Denoised Image (Model 2)")
                axs[2].axis("off")

                plt.show()

test_model(u_net, autoencoder, test_dataloader)
